**SA305 &#x25aa; Linear Programming &#x25aa; Spring 2021 &#x25aa; Uhan**

# Lab 2. Parameterized Optimization Models with Pyomo

⚠️ In order to complete this lab, you need to have Pyomo and GLPK installed on your computer.

## In this lab...

- Lists and dictionaries


- An example: the parameterized linear program for the Farmer Jones problem in Pyomo


- Your assignment

<hr style="border-top: 1px solid gray;"/>

## Lists

- Before we begin, we need to learn a little more about Python's data structures.


- A __list__ is a collection of items that are organized in a particular order.


- You can think of a list as an array or vector.


- For example, here is a list containing the first 5 square numbers:

In [1]:
# Solution
squares = [1, 4, 9, 16, 25]

print(squares)

[1, 4, 9, 16, 25]


- Here is another list containing the days of the week:

In [2]:
# Solution
days_of_the_week = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']

print(days_of_the_week)

['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat']


## Dictionaries

- A __dictionary__ is another way to organize a collection of items.


- A dictionary maps __keys__ to __values__.
    - Just like a real-world dictionary maps _words_ to _definitions_.
    
    
- For example, here is a dictionary that maps English words to their Spanish counterparts:

In [3]:
# Solution
spanish = {'home': 'casa', 'friend': 'amigo', 'navy': 'armada'}

print(spanish)

{'home': 'casa', 'friend': 'amigo', 'navy': 'armada'}


- We can split the definition of a dictionary across multiple lines: like with parentheses `( )`, Python will assume that a statement continues to the next line if it is contained within braces `{ }`, like this:

In [4]:
# Solution
spanish = {
    'home': 'casa', 
    'friend': 'amigo', 
    'navy': 'armada'
}

print(spanish)

{'home': 'casa', 'friend': 'amigo', 'navy': 'armada'}


- We can look up a value in a dictionary like this:

In [5]:
# Solution
print(spanish['navy'])

armada


- Dictionary keys can be multi-dimensional. For example:

In [6]:
# Solution
a_dict = {
    ('eggs', 'chocolate'): 4,
    ('eggs', 'vanilla'): 2,
    ('flour', 'chocolate'): 4,
    ('flour', 'vanilla'): 6
}

print(a_dict['eggs', 'vanilla'])

2


- These multi-dimensional keys are known as __tuples__.

<hr style="border-top: 1px solid gray;"/>

## The Farmer Jones problem and parameterized model

* Recall the Farmer Jones problem from Lessons 2 and 7:

__Example (Rader Example 1.1, modified).__ Farmer Jones decides to supplement her income by baking and selling two types of cakes, chocolate and vanilla. Each chocolate cake sold gives a profit of \\$3, and the profit on each vanilla cake sold is \\$4. Each chocolate cake uses 4 eggs and 4 pounds of flour, while each vanilla cake uses 2 eggs and 6 pounds of flour. If Farmer Jones has only 32 eggs and 48 pounds of flour available, how many of each type of cake should Farmer Jones bake in order to maximize her profit? For now, assume all cakes baked are sold, and fractional cakes are OK.

- In Lesson 7, we came up with the following parameterized linear program for Farmer Jones's problem:

__Sets.__

\begin{align*}
    K & = \text{set of cake types}\\
    I & = \text{set of ingredients used}
\end{align*}

__Parameters.__

\begin{alignat*}{2}
    p_k & = \text{unit profit for type $k$ cakes} &\quad& \text{for } k \in K\\
    b_i & = \text{units of ingredient $i$ available} &\quad& \text{for } i \in I\\
    a_{i,k} & = \text{units of ingredient $i$ used in cake type $k$} &\quad& \text{for } i \in I, k \in K    
\end{alignat*}

__Decision variables.__

\begin{equation*}
    x_k = \text{number of type $k$ cakes to bake} \quad \text{for } k \in K
\end{equation*}

__Objective function and constraints.__

\begin{alignat*}{2}
\text{maximize} \quad & \sum_{k \in K} p_k x_k &\qquad& \text{(maximize profit)}\\
\text{subject to} \quad & \sum_{k \in K} a_{i,k} x_k \le b_i \quad \text{for } i \in I &\qquad& \text{(ingredients available)}\\
& x_k \ge 0 \quad \text{for } k \in K &\qquad& \text{(nonnegativity)}
\end{alignat*}

- For this example, the sets and parameters above have the following concrete values:

\begin{align*}
K & = \{ \text{chocolate}, \text{vanilla} \}\\
I & = \{ \text{eggs}, \text{flour} \}
\end{align*}

\begin{align*}
p_{\text{chocolate}} & = 3 & a_{\text{eggs}, \text{chocolate}} & = 4 & a_{\text{eggs}, \text{vanilla}} & = 2\\
p_{\text{vanilla}} & = 4 & a_{\text{flour}, \text{chocolate}} & = 4 & a_{\text{flour}, \text{vanilla}} & = 6\\
& & b_{\text{eggs}} & = 32 & b_{\text{flour}} & = 48
\end{align*}

* We can construct and solve this model with Pyomo with the steps below.

### 1. Import Pyomo.

- We begin by importing the Pyomo library, just like in Lab 1:

In [7]:
import pyomo.environ as pyo

### 2. Define lists and dictionaries that define concrete values for the sets and parameters.

- Before creating the Pyomo model, we define lists and dictionaries that define concrete values for the sets and parameters in our parameterized model.

In [8]:
# Solution
K_list = ['chocolate', 'vanilla']
I_list = ['eggs', 'flour']

p_dict = {
    'chocolate': 3,
    'vanilla': 4
}

b_dict = {
    'eggs': 32,
    'flour': 48
}

a_dict = {
    ('eggs', 'chocolate'): 4,
    ('eggs', 'vanilla'): 2,
    ('flour', 'chocolate'): 4,
    ('flour', 'vanilla'): 6
}

### 3. Initialize a concrete model.

- Next, we initialize a Pyomo concrete model in the variable `model`.

In [9]:
# Solution
model = pyo.ConcreteModel()

### 4. Define and initialize the sets and parameters.

- The model we want to construct has the following sets:

\begin{align*}
    K & = \text{set of cake types}\\
    I & = \text{set of ingredients used}
\end{align*}

- We can add these sets to our model in Pyomo and initialize their values with the lists we created in Step 2 as follows:    

In [10]:
# Solution
model.K = pyo.Set(initialize=K_list)
model.I = pyo.Set(initialize=I_list)

- We also want to add the following parameters to our model:

\begin{alignat*}{2}
    p_k & = \text{unit profit for type $k$ cakes} &\quad& \text{for } k \in K\\
    b_i & = \text{units of ingredient $i$ available} &\quad& \text{for } i \in I\\        
    a_{i,k} & = \text{units of ingredient $i$ used in cake type $k$} &\quad& \text{for } i \in I, k \in K
\end{alignat*}

- We can add these parameters to our model in Pyomo and initialize their values with the dictionaries we created in Step 2 as follows:

In [11]:
# Solution
model.p = pyo.Param(model.K, initialize=p_dict)
model.b = pyo.Param(model.I, initialize=b_dict)
model.a = pyo.Param(model.I, model.K, initialize=a_dict)

- Note that the first arguments to `pyo.Param()` are the sets corresponding to the subscripts/indices of that parameter.

### 5. Define the decision variables and variable bounds.

* The model we want to construct in Pyomo has the following decision variables:

\begin{equation*}
    x_k = \text{number of type $k$ cakes to bake} \quad \text{for } k \in K
\end{equation*}

* The variable bounds are:

\begin{equation*}
    x_k \ge 0 \quad \text{for } k \in K \qquad \text{(nonnegativity)}
\end{equation*}

* We can add these to our model in Pyomo as follows:

In [12]:
# Solution
model.x = pyo.Var(model.K, domain=pyo.NonNegativeReals)

- Like with `pyo.Param()`, the first arguments are the subscript/index sets for the decision variables.


- As we saw in Lab 1, the keyword argument `domain=...` lets us specify the variable domain.

### 6. Define the objective function.

- The objective function we want to add to our model in Pyomo is:

\begin{equation*}
\text{maximize} \quad  \sum_{k \in K} p_k x_k \qquad \text{(maximize profit)}
\end{equation*}

- We can do so like this:

In [13]:
# Solution
def obj_rule(model):
    return sum(model.p[k] * model.x[k] for k in model.K)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

- 👋 Note that the sum
    \begin{equation*}
        \sum_{k \in K} p_k x_k
    \end{equation*}

    is written as
    
    ```python
    sum(model.p[k] * model.x[k] for k in model.K)
    ```
    
    in the `return` statement of the function `obj_rule()`.

### 7. Define the constraints.

- The ingredient availability constraints are:

    \begin{equation*}
    \sum_{k \in K} a_{i,k} x_k \le b_i \quad \text{for } i \in I \qquad \text{(ingredients available)}
    \end{equation*}


- We can add these constraints to our model in Pyomo as follows:

In [14]:
# Solution
def ingredients_available_rule(model, i):
    return sum(model.a[i,k] * model.x[k] for k in model.K) <= model.b[i]

model.ingredients_available = pyo.Constraint(model.I, rule=ingredients_available_rule)

- 👋 Note that `ingredients_available_rule()` now has 2 arguments: 
    - the first argument is the model, and 
    - the second argument is the index corresponding to the for statement of these constraints
    
    
- 👋 Also note that the first argument to `pyo.Constraint()` is the index corresponding to the for statement of these constraints

### 8. Solve the model.

- This works the same way as before:

In [15]:
result = pyo.SolverFactory("glpk").solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpvohe50be.glpk.raw
 --wglp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpvvd0o1a4.glpk.glp
 --cpxlp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmp34o38yzj.pyomo.lp
Reading problem data from '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmp34o38yzj.pyomo.lp'...
3 rows, 3 columns, 5 non-zeros
26 lines were read
Writing problem data to '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpvvd0o1a4.glpk.glp'...
19 lines were written
GLPK Simplex Optimizer, v4.65
3 rows, 3 columns, 5 non-zeros
Preprocessing...
2 rows, 2 columns, 4 non-zeros
Scaling...
 A: min|aij| =  2.000e+00  max|aij| =  6.000e+00  ratio =  3.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 2
*     0: obj =  -0.000000000e+00 inf =   0.000e+00 (2)
*     2: obj =   3.400000000e+01 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION 

### 9. Determine the status.

- As before, we can directly access the status of the solving process like this:

In [16]:
# Solution
print(f'The solver returned a status of: {result.solver.termination_condition}')

The solver returned a status of: optimal


### 10. Print the optimal solution and its value if one exists.

- Also as before, we can use an `if` statement to check if the solver terminated with an optimal solution, and print the optimal value.


- Since our decision variables are defined for each element of a set, we can use a `for` loop to print the value of each decision variable:

In [17]:
# Solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal value: {pyo.value(model.obj)}")
    print("Optimal solution:")
    
    for k in model.K:
        print(f"  x[{k}] = {pyo.value(model.x[k])}")

Optimal value: 34.0
Optimal solution:
  x[chocolate] = 6.0
  x[vanilla] = 4.0


### The value of parameterization 🤔

- As we discussed in class, the parameterized optimization model is __valid for any problem of the same structure__.


- If the cake types, ingredients, or recipes change, __all we need to do is change the lists and dictionaries we defined in Step 2__. All the other steps remain exactly the same!

<hr style="border-top: 1px solid gray;"/>

## Your assignment

Recall Exercise 2.13 from Rader (the Midwest Steel problem), assigned for homework. We can formulate this problem as a linear program as follows.

__Sets.__

\begin{alignat*}{2}
  R & = \text{set of raw materials} = \{ \text{Alloy1, Alloy2, Alloy3, Scrap1, Scrap2} \}\\ 
  C & = \text{set of characteristics} = \{\text{Carbon, Nickel, Chromium, TensileStrength}\}
\end{alignat*}

__Parameters.__

\begin{alignat*}{2}
  c_i & = \text{cost of raw material $i$} && \text{for } i \in R\\
  a_i & = \text{amount of raw material $i$ available} && \text{for } i \in R\\
  h_{ij} & = \text{value of characteristic $j$ in raw material $i$} &\quad& \text{for } i \in R, j \in C\\
  \ell_j & = \text{lower bound for characteristic $j$} && \text{for } j \in C\\
  u_j & = \text{upper bound for characteristic $j$} && \text{for } j \in C
\end{alignat*}

__Decision variables.__

\begin{equation*}
    x_i = \text{tons of raw material $i$ used} \quad \text{for } i \in R
\end{equation*}

__Objective function and constraints.__

\begin{alignat*}{3}
\min \quad & \sum_{i \in R} c_i x_i &\quad& &\quad& \text{(total cost)}\\ 
\text{s.t.} \quad & \sum_{i \in R} x_i = 100 &\quad&  &\quad& \text{(piece is 100 tons)}\\
& \ell_j \sum_{i \in R} x_i \leq \sum_{i \in R} h_{ij} x_i \leq u_j \sum_{i \in R} x_i &\quad& \text{for $j \in C$} &\quad& \text{(characteristic requirements)}\\
& 0 \leq x_i \leq a_i &\quad& \text{for $i \in R$} &\quad& \text{(nonnegativity, availability of raw materials)}
\end{alignat*}

The sets above have the following concrete values:

\begin{align*}
R & = \{ \text{Alloy1, Alloy2, Alloy3, Scrap1, Scrap2} \}\\ 
C & = \{\text{Carbon, Nickel, Chromium, TensileStrength}\}
\end{align*}

The parameters above have the following concrete values:

| $i \in R$ | $c_i$ | $a_i$ |
| :- | -: | -: |
| Alloy1 | 150 | 50 |
| Alloy2 | 120 | 50 |
| Alloy3 | 80 | 20 |
| Scrap1 | 35 | 30 |
| Scrap2 | 20 | 40 |

| $j \in C$ | $\ell_j$ | $u_j$ |
| :- | -: | -: |
| Carbon | 0.020 | 0.030 |
| Nickel | 0.000 | 0.040 | 
| Chromium | 0.013 | 0.027 |
| TensileStrength | 50000 | 80000 |

| $h_{ij}$ | Carbon | Nickel | Chromium | TensileStrength |
| :- | -: | -: | -: | -: |
| Alloy1 | 0.0175 | 0.020 | 0.035 | 60000 |
| Alloy2 | 0.0245 | 0.030 | 0.008 | 40000 |
| Alloy3 | 0.0280 | 0.040 | 0.012 | 90000 |
| Scrap1 | 0.0310 | 0.045 | 0.039 | 120000 |
| Scrap2 | 0.0350 | 0.055 | 0.028 | 70000 |

Construct and solve this model in Pyomo by completing the tasks given below.

__1.__  Import the Pyomo library.

In [18]:
# Solution
import pyomo.environ as pyo

__2a.__ Define lists `R_list` and `C_list` that define concrete values for the sets $R$ and $C$, respectively.

In [19]:
# Solution
R_list = ['Alloy1', 'Alloy2', 'Alloy3', 'Scrap1', 'Scrap2']
C_list = ['Carbon', 'Nickel', 'Chromium', 'TensileStrength']

__2b.__ Define dictionaries `c_dict` and `a_dict` that define concrete values for the parameters $c_i$ and $a_i$ for $i \in R$, respectively.

In [20]:
# Solution
c_dict = {
    'Alloy1': 150,
    'Alloy2': 120,
    'Alloy3': 80,
    'Scrap1': 35,
    'Scrap2': 20
}

a_dict = {
    'Alloy1': 50,
    'Alloy2': 50,
    'Alloy3': 20,
    'Scrap1': 30,
    'Scrap2': 40
}

__2c.__ Define dictionaries `l_dict` and `u_dict` that define concrete values for the parameters $\ell_j$ and $u_j$ for $j \in C$, respectively.

In [21]:
# Solution
l_dict = {
    'Carbon': 0.020,
    'Nickel': 0.000,
    'Chromium': 0.013,
    'TensileStrength': 50000
}

u_dict = {
    'Carbon': 0.030,
    'Nickel': 0.040,
    'Chromium': 0.027,
    'TensileStrength': 80000
}

__2d.__ Define dictionary `h_dict` that defines concrete values for the parameters $h_{ij}$ for $i \in R$ and $j \in C$. Use multidimemsional (tuple) keys.

In [22]:
# Solution
h_dict = {
    ('Alloy1', 'Carbon'): 0.0175,
    ('Alloy1', 'Nickel'): 0.020,
    ('Alloy1', 'Chromium'): 0.035,
    ('Alloy1', 'TensileStrength'): 60000,    
    ('Alloy2', 'Carbon'): 0.0245,
    ('Alloy2', 'Nickel'): 0.030,
    ('Alloy2', 'Chromium'): 0.008,
    ('Alloy2', 'TensileStrength'): 40000,     
    ('Alloy3', 'Carbon'): 0.0280,
    ('Alloy3', 'Nickel'): 0.040,
    ('Alloy3', 'Chromium'): 0.012,
    ('Alloy3', 'TensileStrength'): 90000,     
    ('Scrap1', 'Carbon'): 0.0310,
    ('Scrap1', 'Nickel'): 0.045,
    ('Scrap1', 'Chromium'): 0.039,
    ('Scrap1', 'TensileStrength'): 120000,     
    ('Scrap2', 'Carbon'): 0.0350,
    ('Scrap2', 'Nickel'): 0.055,
    ('Scrap2', 'Chromium'): 0.028,
    ('Scrap2', 'TensileStrength'): 70000
}

__3.__ Initialize a concrete model named `model`.

In [23]:
# Solution
model = pyo.ConcreteModel()

__4a.__ Define and initialize the sets $R$ and $C$ as `model.R` and `model.C`, respectively.

In [24]:
# Solution
model.R = pyo.Set(initialize=R_list)
model.C = pyo.Set(initialize=C_list)

__4b.__ Define and initialize the parameters $c_i$ and $a_i$ for $i \in R$, $\ell_j$ and $u_j$ for $j \in C$, and $h_{ij}$ for $i \in R$ and $j \in C$ as `model.c`, `model.a`, `model.l`, `model.u`, and `model.h`, respectively.

In [25]:
# Solution
model.c = pyo.Param(model.R, initialize=c_dict)
model.a = pyo.Param(model.R, initialize=a_dict)
model.l = pyo.Param(model.C, initialize=l_dict)
model.u = pyo.Param(model.C, initialize=u_dict)
model.h = pyo.Param(model.R, model.C, initialize=h_dict)

__5.__ Define the decision variables $x_i$ for $i \in R$ as `model.x`.  Make sure you specify their nonnegativity. We'll tackle their upper bounds later as constraints.

In [26]:
# Solution
model.x = pyo.Var(model.R, domain=pyo.NonNegativeReals)

__6.__ Define the objective function as `model.obj`.  Make sure you specify the correct `sense`.

In [27]:
# Solution
def obj_rule(model):
    return sum(model.c[i] * model.x[i] for i in model.R)

model.obj = pyo.Objective(rule=obj_rule, sense=pyo.minimize)

__7a.__ Define the "piece is 100 tons" constraint as `model.piece`.

In [28]:
# Solution
def piece_rule(model):
    return sum(model.x[i] for i in model.R) == 100

model.piece = pyo.Constraint(rule=piece_rule)

⚠️ Next, in __7b__ and __7c__, we split the characteristic requirement constraints into two separate sets of inequality constraints.

__7b.__ Define the characteristic requirement lower bound constraints

\begin{equation*}
\ell_j \sum_{i \in R} x_i \leq \sum_{i \in R} h_{ij} x_i \quad \text{for $j \in C$}
\end{equation*}

as `model.characteristic_lower`. 

In [29]:
# Solution
def characteristic_lower_rule(model, j):
    return (
        model.l[j] * sum(model.x[i] for i in model.R) 
        <= sum(model.h[i,j] * model.x[i] for i in model.R)
    )

model.characteristic_lower = pyo.Constraint(
    model.C, rule=characteristic_lower_rule
)

__7c.__ Define the characteristic requirement upper bound constraints

\begin{equation*}
\sum_{i \in R} h_{ij} x_i \leq u_j \sum_{i \in R} x_i \quad \text{for $j \in C$}
\end{equation*}

as `model.characteristic_upper`. 

In [30]:
# Solution
def characteristic_upper_rule(model, j):
    return (
        sum(model.h[i,j] * model.x[i] for i in model.R)
        <= model.u[j] * sum(model.x[i] for i in model.R)
    )

model.characteristic_upper = pyo.Constraint(
    model.C, rule=characteristic_upper_rule
)

__7d.__ Define the availability constraints

\begin{equation*}
x_i \leq a_i \quad \text{for $i \in R$}
\end{equation*}

as `model.availability`. 

In [31]:
# Solution
def availability_rule(model, i):
    return model.x[i] <= model.a[i]

model.availability = pyo.Constraint(
    model.R, rule=availability_rule
)

__8.__ Solve the model and save the result as `result`.

In [32]:
# Solution
result = pyo.SolverFactory('glpk').solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmptp7gilb7.glpk.raw
 --wglp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpkobnrc04.glpk.glp
 --cpxlp /var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpqz0nixsk.pyomo.lp
Reading problem data from '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpqz0nixsk.pyomo.lp'...
15 rows, 6 columns, 50 non-zeros
113 lines were read
Writing problem data to '/var/folders/5k/rxb0jk152pb3hcrczw45mrm40000gn/T/tmpkobnrc04.glpk.glp'...
94 lines were written
GLPK Simplex Optimizer, v4.65
15 rows, 6 columns, 50 non-zeros
Preprocessing...
8 rows, 5 columns, 39 non-zeros
Scaling...
 A: min|aij| =  1.000e-03  max|aij| =  7.000e+04  ratio =  7.000e+07
GM: min|aij| =  2.474e-01  max|aij| =  4.041e+00  ratio =  1.633e+01
EQ: min|aij| =  6.507e-02  max|aij| =  1.000e+00  ratio =  1.537e+01
Constructing initial basis...
Size of triangular part is 8
      0: obj =   2.000

__9.__ Print the status of the solving process of the model.

In [33]:
# Solution
print(f'The solver returned a status of: {result.solver.termination_condition}')

The solver returned a status of: optimal


__10.__ If the solver terminated with an optimal solution, print the optimal solution and its value.

In [34]:
# Solution
if result.solver.termination_condition == pyo.TerminationCondition.optimal:
    print(f"Optimal value: {pyo.value(model.obj)}")
   
    print("Optimal solution:")
    for i in model.R:
        print(f"  x[{i}] = {pyo.value(model.x[i])}")

Optimal value: 7086.076817558309
Optimal solution:
  x[Alloy1] = 16.3923182441701
  x[Alloy2] = 18.4910836762689
  x[Alloy3] = 10.9327846364884
  x[Scrap1] = 30.0
  x[Scrap2] = 24.1838134430727


<hr style="border-top: 1px solid gray;"/>

## When you're finished

- Make sure your notebook runs from top to bottom with no errors. One way to accomplish this is to click on __Kernel &#8594; Restart & Run All__. This will restart Python, and run your notebook from top to bottom.

<hr style="border-top: 1px solid gray;"/>

## Grading

| Problem | Weight |
| - | - |
| 1 | 0.5 |
| 2a | 0.5 |
| 2b | 0.5 | 
| 2c | 0.5 | 
| 2d | 0.5 |
| 3 | 0.5 |
| 4a | 0.5 |
| 4b | 0.5 |
| 5 | 1 |
| 6 | 1 |
| 7a | 1 |
| 7b | 1 |
| 7c | 1 | 
| 7d | 1 |
| 8 | 0.5 |
| 9 | 0.5 |
| 10 | 1 |
| __Total__ | __120 points__ |